In [ ]:
import os
import cv2
videos = ["_".join(i.split("_")[:-1]) for i in os.listdir("sim/in")]

frames = sorted([i for i in os.listdir("sim/in") if i.startswith("vid2_human")])

In [3]:
# import numpy as np
# from IPython.display import display, clear_output, HTML
# import cv2
# import pylab


# bgsub = cv2.createBackgroundSubtractorMOG2(history=10)
# for frame in frames:
#     img = cv2.imread("sim/in/" + frame)
#     bgsub.apply(img)
#     bg = bgsub.getBackgroundImage().astype(float)
#     diff = cv2.absdiff(img.astype(float), bg)/255 * 10
#     diff = np.clip(diff, 0, 1)
#     pylab.imshow(diff)
#     pylab.axis('off')
#     display(pylab.gcf())
#     clear_output(wait=True)

In [4]:
# import requests

# import torch
# from PIL import Image
# from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

# model_id = "IDEA-Research/grounding-dino-tiny"
# device = "cuda"

# processor = AutoProcessor.from_pretrained(model_id)
# model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

from groundingdino.util.inference import load_model, load_image, predict
import cv2
model = load_model("GroundingDINO/groundingdino/config/GroundingDINO_SwinB_cfg.py", "GroundingDINO/groundingdino_swinb_cogcoor.pth")


final text_encoder_type: bert-base-uncased


In [13]:
import pylab
import numpy as np
from IPython.display import display, clear_output, HTML
from torchvision.ops import box_convert
import torch
from tqdm import tqdm

# text = "steam. human. birds. cars. "
text = "on a black background, there are possible white bird and white car or white human. There could also be possible a plume of gas, which look like white semi-transparent fluid."
# white car so it udnerstand the color
img0 = cv2.imread("sim/in/" + frames[0])
history_state = []
bgsub = cv2.createBackgroundSubtractorMOG2(history=30)
video_writer = cv2.VideoWriter("out.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (img0.shape[1], img0.shape[0]))
for frame in tqdm(frames):
    img = cv2.imread("sim/in/" + frame)
    bgsub.apply(img)
    bg = bgsub.getBackgroundImage().astype(float)
    diff = cv2.absdiff(img.astype(float), bg) #too large cannot see detial
    diff = np.clip(diff, 0, 255).astype(np.uint8)
    
    cv2.imwrite("/tmp/frame.png", diff)
    image_source, image = load_image("/tmp/frame.png")   
    h, w, _ = image_source.shape
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=text,
        box_threshold=0.3,
        text_threshold=0.3
    )

    frame = image_source.copy()
    current_state = []
    for box, phrase, logit in zip(boxes, phrases, logits):
        # convert box (0-1) from xywh to pixel in x1, y1, x2, y2
        boxes_unnorm = box * torch.tensor([w, h, w, h], dtype=torch.float32)
        boxes_unnorm = box_convert(boxes_unnorm.unsqueeze(0), 'cxcywh', 'xyxy').squeeze(0).int().tolist()
        x1, y1, x2, y2 = boxes_unnorm
        if "plume" in phrase or "steam" in phrase:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0,255), 2)
            cv2.putText(frame, phrase, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,255), 1)
            current_state.append((x1, y1, x2, y2))
        else:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, phrase, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 1)
    history_state.append(current_state)
    video_writer.write(frame)
    
video_writer.release()
os.system("ffmpeg -i out.mp4 -c:v libx264 out_x264.mp4 -y > /dev/null 2>&1")

  0%|          | 0/451 [00:00<?, ?it/s]

100%|██████████| 451/451 [01:27<00:00,  5.13it/s]


0